In [ ]:
"""
Electrolyte Optimization Demo
Each dataset corresponds to one iteration of the closed-loop optimization process (D0, D1, D2, ...). 
The column "cycle-life" is derived following the procedure described in Supplementary Fig. 23.
"""

import pandas as pd
import numpy as np

# === Read input data from iteration D3 ===
data = pd.read_csv("data/D3_data.csv", encoding="gbk", header=None)

# Rename columns for clarity
data.columns = ["EncodedFormulation", "CycleLife"]

print("Number of entries:", len(data))
print(data.head())


Number of entries: 99
  EncodedFormulation  CycleLife
0               1Ob2  18.000000
1               2Lc1  63.898305
2               1Ib1  16.636364
3               1Aa1  64.257143
4               4Oa2  10.789474


In [2]:
# Mapping dictionaries for each position in the code
salt_map = {
    '1': 'LiPF6',
    '2': 'LiTFSI',
    '3': 'LiFSI',
    '4': 'LiDFOB'
}

solvent_map = {
    'A': ('EC', 'DMC'),
    'B': ('EC', 'DME'),
    'C': ('EC', 'DOL'),
    'D': ('EC', 'ES'),
    'E': ('EC', 'PC'),
    'F': ('DMC', 'DME'),
    'G': ('DMC', 'DOL'),
    'H': ('DMC', 'ES'),
    'I': ('DMC', 'PC'),
    'J': ('DME', 'DOL'),
    'K': ('DME', 'ES'),
    'L': ('DME', 'PC'),
    'M': ('DOL', 'ES'),
    'N': ('DOL', 'PC'),
    'O': ('ES', 'PC')
}

additive_map = {
    'a': 'None',
    'b': 'VC',
    'c': 'FEC',
    'd': 'LiNO3'
}

concentration_map = {
    '1': '1m',
    '2': '2m',
    '5': '5m'
}

# Parse the encoded data
results = []
for i in range(len(data)):
    code = str(data.iloc[i, 0])  # e.g., "1A10"
    
    # Decode each position according to predefined maps
    x0 = salt_map.get(code[0], "Unknown")            # Salt type
    x1, x2 = solvent_map.get(code[1], ("Unknown", "Unknown"))  # Solvent pair
    x3 = additive_map.get(code[2], "Unknown")        # Additive
    x4 = concentration_map.get(code[3], "Unknown")   # Concentration
    
    results.append([x0, x1, x2, x3, x4, data.iloc[i, 1]])

# Convert to DataFrame for easier inspection
df_results = pd.DataFrame(results, columns=["Salt", "Solvent1", "Solvent2", "Additive", "Concentration", "cycle_life"])
df_results.head()


,Salt,Solvent1,Solvent2,Additive,Concentration,cycle_life
0,LiPF6,ES,PC,VC,2m,18.000000
1,LiTFSI,DME,PC,FEC,1m,63.898305
2,LiPF6,DMC,PC,VC,1m,16.636364
3,LiPF6,EC,DMC,None,1m,64.257143
4,LiDFOB,ES,PC,None,2m,10.789474


### Target statistics coding (TSC)

To quantify correlation information among electrolyte components, 
we transform categorical descriptors into numerical features using 
**target statistics coding (TSC)**. 

Following the formulation:

$$
\phi(X_i^{(k)}) = E(y | X_i = X_i^{(k)})
= \frac{\sum_{j=1}^N \mathbf{1}_{X_i^{(j)} = X_i^{(k)}} \cdot y_j}
       {\sum_{j=1}^N \mathbf{1}_{X_i^{(j)} = X_i^{(k)}}}
$$

where $y_j$ is the target value (cycle life in this work). Since this estimation may be unreliable for low-frequency features, a prior $p$ is introduced for smoothing, formulated as:

$$
\phi(X_i^{(k)}) = \lambda_i \cdot E(y|X_i=X_i^{(k)}) + (1-\lambda_i) \cdot p
$$

with $p$ as the global prior and $\lambda_i$ a confidence weight.  

---

More generally, for the *k*-th sample’s electrolyte formulation composed of $L$ distinct 
components, the feature vector is represented as:

$$
X^{(k)} = [ \langle x_l^{(k)}, x_s^{(k)} \rangle, \dots, \langle x_1^{(k)}, x_2^{(k)}, \dots, x_L^{(k)} \rangle ], 
\quad 1 \leq l,s \leq L \tag{12}
$$

and the corresponding relational features are quantified as:

$$
\phi(X^{(k)}) = [ \phi(\langle x_l^{(k)}, x_s^{(k)} \rangle), \dots, 
\phi(\langle x_1^{(k)}, x_2^{(k)}, \dots, x_L^{(k)} \rangle) ], 
\quad 1 \leq l,s \leq L \tag{13}
$$

---

As an example, here we construct a **5-dimensional input vector** for each electrolyte:  

$$
X^{(k)} = [ \ \phi(\langle Salt, Solvent1 \rangle),\ 
             \phi(\langle Salt, Solvent2 \rangle),\ 
             \phi(\langle Solvent1, Solvent2 \rangle),\ 
             \phi(Additive),\ 
             \phi(Concentration) \ ]
$$

This demonstrates how TSC maps categorical features into 
numerical representations while preserving component correlation information.  


In [3]:
def lamb(n, k=5, f=2):
    """ Smoothing factor λ(n) used in TSC encoding """
    return 1 / (1 + np.exp(-(n - k) / f))

# --- Additive encoder ---
def Additive_Targetencoder(df, y):
    """
    Target statistics encoding for additive types.
    Args:
        df : DataFrame with column 'Additive'
        y  : ndarray, target values (cycle life)
    Returns:
        dict : mapping {additive: encoded_value}
    """
    additive = df["Additive"].values
    y_value = np.array(y).reshape(-1)

    groups = {'None': [], 'VC': [], 'FEC': [], 'LiNO3': []}
    for j, a in enumerate(additive):
        if a in groups:
            groups[a].append(y_value[j])

    priori = np.mean(y_value)
    encoders = {}
    for key, vals in groups.items():
        if len(vals) > 0:
            posteriori = np.mean(vals)
            encoders[key] = lamb(len(vals)) * posteriori + (1 - lamb(len(vals))) * priori
        else:
            encoders[key] = priori
    return encoders

# --- Concentration encoder ---
def Concentration_Targetencoder(df, y):
    """
    Target statistics encoding for concentration levels.
    Args:
        df : DataFrame with column 'Concentration'
        y  : ndarray, target values
    Returns:
        dict : mapping {concentration: encoded_value}
    """
    conc = df["Concentration"].values
    y_value = np.array(y).reshape(-1)

    groups = {'1m': [], '2m': [], '5m': []}
    for j, c in enumerate(conc):
        if c in groups:
            groups[c].append(y_value[j])

    priori = np.mean(y_value)
    encoders = {}
    for key, vals in groups.items():
        if len(vals) > 0:
            posteriori = np.mean(vals)
            encoders[key] = lamb(len(vals)) * posteriori + (1 - lamb(len(vals))) * priori
        else:
            encoders[key] = priori
    return encoders

# --- Salt + Solvent encoder (ADI version) ---
from itertools import combinations

def SaltSolvent_ADI(df, y):
    """
    Average degree of interaction (ADI) encoding:
      - Salt + single solvent (24 combinations: 4 × 6)
      - Solvent pairs (15 combinations: C(6,2))
    Args:
        df : DataFrame with columns ['Salt','Solvent1','Solvent2']
        y  : array-like, target values
    Returns:
        dict : mapping {combination: average_value}
    """
    salts = df["Salt"].unique()
    solvents = sorted(set(df["Solvent1"]).union(set(df["Solvent2"])))
    y_value = np.array(y).reshape(-1)

    # collect groups
    groups = {}
    for j in range(len(y_value)):
        salt = df["Salt"].iloc[j]
        s1, s2 = df["Solvent1"].iloc[j], df["Solvent2"].iloc[j]

        # Salt + single solvents
        for s in [s1, s2]:
            key = (salt, s)
            groups.setdefault(key, []).append(y_value[j])

        # solvent pair (no salt)
        if s1 != s2:
            key = frozenset([s1, s2])
            groups.setdefault(key, []).append(y_value[j])

    priori = np.mean(y_value)
    encoders = {}

    # Salt + single solvent (24)
    for salt in salts:
        for s in solvents:
            key = (salt, s)
            vals = groups.get(key, [])
            encoders[f"{salt}+{s}"] = float(np.mean(vals)) if vals else float(priori)

    # Solvent pairs (15)
    for pair in combinations(solvents, 2):
        key = frozenset(pair)
        vals = groups.get(key, [])
        encoders["+".join(sorted(pair))] = float(np.mean(vals)) if vals else float(priori)

    return encoders


In [4]:
# Prepare target values
y = df_results["cycle_life"].values

# Compute encodings
add_enc = Additive_Targetencoder(df_results, y)
conc_enc = Concentration_Targetencoder(df_results, y)
ss_enc   = SaltSolvent_ADI(df_results, y)

# Display results
print("Additive encoding:")
for k,v in add_enc.items():
    print(f"  {k:6s} : {v:.2f}")

print("\nConcentration encoding:")
for k,v in conc_enc.items():
    print(f"  {k:3s} : {v:.2f}")

print("\nSalt+Solvent encoding (first 5 shown):")
for i, (k,v) in enumerate(ss_enc.items()):
    if i >= 5: break
    print(f"  {k:15s} : {v:.2f}")

Additive encoding:
  None   : 63.80
  VC     : 69.74
  FEC    : 70.00
  LiNO3  : 54.33

Concentration encoding:
  1m  : 77.67
  2m  : 66.19
  5m  : 17.50

Salt+Solvent encoding (first 5 shown):
  LiPF6+DMC       : 28.60
  LiPF6+DME       : 51.05
  LiPF6+DOL       : 65.26
  LiPF6+EC        : 65.21
  LiPF6+ES        : 13.28


In [5]:
import itertools

# --- Build 5D feature matrix (based on encoders) ---
def build_feature_matrix(df, add_enc, conc_enc, ss_enc):
    """
    Construct 5D feature vectors:
      [Salt+Solvent1, Salt+Solvent2, SolventPair, Additive, Concentration]
    """
    X = []
    for i in range(len(df)):
        salt = df.iloc[i]["Salt"]
        s1, s2 = df.iloc[i]["Solvent1"], df.iloc[i]["Solvent2"]
        add, conc = df.iloc[i]["Additive"], df.iloc[i]["Concentration"]

        x1 = ss_enc.get(f"{salt}+{s1}", np.mean(list(ss_enc.values())))
        x2 = ss_enc.get(f"{salt}+{s2}", np.mean(list(ss_enc.values())))
        x3 = ss_enc.get("+".join(sorted([s1, s2])), np.mean(list(ss_enc.values())))
        x4 = add_enc.get(add, np.mean(list(add_enc.values())))
        x5 = conc_enc.get(conc, np.mean(list(conc_enc.values())))

        X.append([x1, x2, x3, x4, x5])
    return np.array(X)

# --- Build input (X) and output (y) from current dataset ---
X_data = build_feature_matrix(df_results, add_enc, conc_enc, ss_enc)
y_data = df_results["cycle_life"].values.reshape(-1, 1)

print("Shape of X_data:", X_data.shape)
print("Shape of y_data:", y_data.shape)
print("Example feature vector (first row):", X_data[0])

# --- Build the full design space (720 combinations) ---
salts = ["LiPF6", "LiTFSI", "LiFSI", "LiDFOB"]
solvents = ["EC", "DMC", "DME", "DOL", "ES", "PC"]
additives = ["None", "VC", "FEC", "LiNO3"]
concentrations = ["1m", "2m", "5m"]

# Enumerate all possible combinations
solvent_pairs = list(itertools.combinations(solvents, 2))
param_space = list(itertools.product(salts, solvent_pairs, additives, concentrations))

df_space = pd.DataFrame(param_space, columns=["Salt", "SolventPair", "Additive", "Concentration"])
df_space["Solvent1"] = df_space["SolventPair"].apply(lambda x: x[0])
df_space["Solvent2"] = df_space["SolventPair"].apply(lambda x: x[1])
df_space = df_space.drop(columns=["SolventPair"])

print("\nTotal design space size:", len(df_space))

# --- Encode the full design space into 5D features ---
X_space = build_feature_matrix(df_space, add_enc, conc_enc, ss_enc)
print("Shape of X_space:", X_space.shape)


Shape of X_data: (99, 5)
Shape of y_data: (99, 1)
Example feature vector (first row): [13.28205128 16.54545455 14.63659148 69.7430646  66.19111029]

Total design space size: 720
Shape of X_space: (720, 5)


### Deep active learning with Deep Kernel Learning (DKL)

For electrolyte optimization, we propose a **deep kernel learning (DKL)-based active learning (AL)** framework.  
This framework is particularly suitable for the electrolyte design problem in lithium metal batteries (LMBs), where the objective function is a black-box mapping from electrolyte formulas to cycle life and is both expensive and noisy to evaluate.  

In DKL, the Gaussian process (GP) is combined with a neural network to enhance representational power.  
Given a dataset with variables $x$ (electrolyte formulas) and $y$ (cell cycle life), the kernel is defined as:

$$
k_{\text{DKL}}(x^{(i)},x^{(j)} \mid \omega,\theta) =
k_{\text{base}}(g(x^{(i)} \mid \omega), g(x^{(j)} \mid \omega) \mid \theta)
$$

where $k_{\text{base}}$ is the base kernel function, $\theta$ its hyperparameters, and $g(\cdot \mid \omega)$ the nonlinear mapping learned by the neural network with weights $\omega$.  
This deep kernel captures complex and non-stationary patterns in electrolyte data, outperforming conventional kernels such as RBF.  

For the acquisition function, we employ **Thompson Sampling (TS)**.  
At iteration $t$, a function $h(x)$ is sampled from the posterior predictive distribution, and the next candidate is chosen as:

$$
x^{(t)} = \arg\max_x h(x), \quad h(x) \sim p(y \mid D_t)
$$

where $D_t=\{(x^{(i)},y^{(i)})\}_{i=1}^{t-1}$ is the set of evaluated data points.  
By iteratively sampling and updating, TS balances exploration and exploitation, accelerating the discovery of optimal electrolyte formulations.


In [6]:
import csv
import random
from dknet import NNRegressor
from dknet.layers import Dense, CovMat, Scale
from dknet.optimizers import Adam

def run_DKL(train_x, train_y, test_x, df_space, batch_size=32):
    """
    Deep Kernel Learning (DKL) with Parallel Thompson Sampling (q-TS).
    """
    # Define DKL model
    layers = [
        Dense(64, activation='tanh'),
        Dense(5),  # latent dim
        Scale(fixed=True, init_vals=64.0),
        CovMat(kernel='rbf', alpha_fixed=False)
    ]
    opt = Adam(1e-3)
    gp = NNRegressor(
        layers,
        opt=opt,
        batch_size=train_x.shape[0],
        maxiter=5000,
        gp=True,
        verbose=False
    )

    # Train GP
    gp.fit(train_x, train_y)

    # Predict posterior mean and std (for logging)
    mu, std, _ = gp.predict(test_x)

    # --- Parallel Thompson Sampling ---
    use_sample_y = True
    tau = 3.0
    selected_idx = []
    for b in range(batch_size * 2):  # oversample for diversity
        if use_sample_y:
            # Full covariance sampling
            sample = gp.sample_y(test_x, n_samples=1, random_state=b).ravel()
        else:
            # Diagonal approximation with exploration temperature
            noise = np.random.randn(len(mu))
            sample = mu.flatten() + noise * (tau * std.flatten())

        max_idx = np.argmax(sample)
        if max_idx not in selected_idx:
            selected_idx.append(max_idx)
        if len(selected_idx) >= batch_size:
            break
    if len(selected_idx) < batch_size:
        if use_sample_y:
            sample = gp.sample_y(test_x, n_samples=1, random_state=999).ravel()
        else:
            noise = np.random.randn(len(mu))
            sample = mu.flatten() + noise * (tau * std.flatten())

        top_idx = np.argsort(sample)[::-1]
        for idx in top_idx:
            if idx not in selected_idx:
                selected_idx.append(idx)
            if len(selected_idx) >= batch_size:
                break


    # Build results DataFrame
    results = []
    for idx in selected_idx:
        row = df_space.iloc[idx].to_dict()
        row["pred_mean"] = float(mu[idx][0])
        row["pred_std"] = float(std[idx])
        results.append(row)

    return pd.DataFrame(results)

# --- Example run ---
df_next = run_DKL(X_data, y_data, X_space, df_space, batch_size=32)
print("Next batch of 32 recommended formulations:")
print(df_next)



Next batch of 32 recommended formulations:
      Salt Additive Concentration Solvent1 Solvent2   pred_mean  pred_std
0    LiFSI    LiNO3            1m      DME      DOL  309.215643  1.353912
1    LiFSI     None            2m      DME      DOL  246.022946  1.190420
2   LiTFSI       VC            2m      DME       ES  236.183560  1.375518
3   LiDFOB     None            2m      DME       ES  236.647931  1.375518
4   LiTFSI     None            2m      DME      DOL  213.000126  1.370667
5   LiTFSI       VC            1m      DME       ES  205.202641  1.110901
6    LiFSI       VC            1m      DME       ES  191.815203  1.364473
7   LiTFSI      FEC            1m      DME       ES  190.370543  1.087233
8   LiTFSI      FEC            1m       EC      DME  167.892293  1.370828
9    LiFSI      FEC            1m       EC      DME  161.467928  1.327252
10   LiFSI      FEC            1m      DME       ES  157.102842  1.364473
11  LiTFSI      FEC            1m      DME      DOL  153.107704  1.14